In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, set_seed

import torch

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
MODEL_NAME = "microsoft/Phi-3-mini-4k-instruct"

model = AutoModelForCausalLM.from_pretrained( 
    MODEL_NAME,  
    device_map=device,  
    torch_dtype="auto",  
    trust_remote_code=True,  
) 

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

In [6]:
template = """Below is a an example of aspect-based sentiment analysis performed on a product review of a telephone. Then there is a list of Aspect Categories: each category is a high level feature of interest to telephone company executives, such as "price" or "usability". Finally there is a query which consists of an actual telephone product review. Perform the same aspect-based sentiment analysis as shown in the example, and follow the exact same output formatting without deviation.

## EXAMPLE OF ASPECT-BASED SENTIMENT ANALYSIS
{example_absa}

## LIST OF ASPECT CATEGORIES
- price
- usability
- reliability
- environment friendliness
- aesthetic of design

## QUERY
product review: {query}
absa output:
"""

In [7]:
example_absa = """
product review: I love how easy to use the interface of my new iPhone 4 Pro is. Unfortunately this comes with the downside that the batteries aren't great and often break down at random moments, not to mention that they are made from rare cobalt that destroys the planet! Overall a good phone for this price point.
absa output:
- \{ \"aspect target\": \"iPhone 4 Pro\", \"aspect category\": \"usability\", \"sentiment\": \"positive\" \}
- \{ \"aspect target\": \"the batteries\", \"aspect category\": \"reliability\", \"sentiment\": \"negative\" \}
- \{ \"aspect target\": \"the batteries\", \"aspect category\": \"environment friendliness\", \"sentiment\": \"negative\" \}
- \{ \"aspect target\": \"iPhone 4 Pro\", \"aspect category\": \"price\", \"sentiment\": \"positive\" \}
"""

query = "The new Samsung galaxy is great overall, and really good value for money - I don't like how complicated the keypad is though"

# ---------


prompt = template.format(example_absa=example_absa, query=query)

messages = [ 
    {"role": "system", "content": "You are helpful asssistant that performs aspect-based sentiment analysis on telephone product reviews."}, 
    {"role": "user", "content": prompt}, 
] 

generation_args = { 
    "max_new_tokens": 500, 
    "return_full_text": False, 
    "temperature": 0.0, 
    #"do_sample": False, 
}

In [10]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

In [11]:
output = pipe(messages, **generation_args)

print(output[0]['generated_text'])

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.


 - \{ "aspect target": "Samsung galaxy", "aspect category": "usability", "sentiment": "negative" \}
- \{ "aspect target": "Samsung galaxy", "aspect category": "price", "sentiment": "positive" \}
